In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import cv2
import numpy as np
import csv
from sklearn.utils import shuffle
%matplotlib inline


In [3]:
#datapath = '/home/microway/Shuo/CarND/CarND-BehaviorCloning-Project/data-given/'
data = pd.read_csv('/Users/Shuo/study/Project-predictive_study/Old_experiments/data_2016_1t_dir1.csv',delimiter=',')

In [4]:
def load_data(data,datafolder='/Users/Shuo/Desktop/Data_2016/'):
    X_train = np.zeros((len(data),13,288,9))
    y_train = np.zeros((len(data),15,288,1))
    for i in range(len(data)):
        with open(datafolder+'CSVs_old/'+data['X'][i], 'r') as f:
            X = list(csv.reader(f, delimiter=","))
            X = np.asarray(X)
            channels = np.unique(X[:,0])
            for channel in channels:
                index=X[:,0] == channel
                X_train[i,:,:,int(channel)] = X[index,1:]
        with open(datafolder+'Traffic_CSVs/'+data['y'][i], 'r') as f:
            y = list(csv.reader(f, delimiter=","))
            y = np.asarray(y)
            index=y[:,0] == '0'
            y_train[i,:,:,0] = y[index,1:]
    return X_train,y_train

In [5]:
X_train,y_train = load_data(data)
print(X_train.shape)
print(y_train.shape)

(346, 13, 288, 9)
(346, 15, 288, 1)


In [6]:
def normalization(X_train):
    table = pd.read_csv('/Users/Shuo/study/Project-predictive_study/stats_weather.csv',delimiter=',')
    for i in range(X_train.shape[-1]):
        mean = table['mean'][i]
        std = table['std'][i]
        X_train[:,:,:,i] = (X_train[:,:,:,i]-mean)/std
    return X_train
    
X_train = normalization(X_train)  


In [7]:
from keras.models import Sequential
from keras.layers import Reshape, Dense, Convolution2D, Deconvolution2D, Flatten, Input, Dropout, MaxPooling2D, Activation
from keras.models import model_from_json
from keras.activations import relu, softmax, linear
from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization


Using TensorFlow backend.


In [ ]:
# Structure 1
# Epoch 10/10 311/311 [==============================] - 9s - loss: 1364.2763 - val_loss: 1280.2520
model = Sequential()
# 13*288*9
model.add(Convolution2D(16, 3, 5,input_shape=(13, 288, 9),border_mode='valid'))
model.add(Activation('relu'))
# 11*284*16
model.add(Convolution2D(32, 3, 5,border_mode='valid'))
model.add(Activation('relu'))
# 9*280*24
model.add(Flatten())
model.add(Dense(512))
model.add(Dense(128))
model.add(Dense(1024))
model.add(Dense(60480))
model.add(Reshape((9,280,24)))
model.add(Deconvolution2D(16, 3, 5, output_shape=(None, 11, 284,16),border_mode='valid'))
model.add(Activation('relu'))
# 11*284*16 
model.add(Deconvolution2D(1, 5, 5, output_shape=(None, 15, 288, 1),border_mode='valid'))

model.compile(loss='mse', optimizer='adam')

In [136]:
# Structure 2
acti = 'relu'

model = Sequential()
# 13*288*9
model.add(Convolution2D(16, 3, 5,input_shape=(13, 288, 9),border_mode='valid',subsample=(1, 1)))
model.add(BatchNormalization())
model.add(Activation(acti))
#model.add(Dropout(0.5))
# 11*284*16
model.add(Convolution2D(24, 3, 5,border_mode='valid',subsample=(1, 1)))
model.add(BatchNormalization())
model.add(Activation(acti))
#model.add(Dropout(0.5))
# 9*280*24
model.add(Convolution2D(32, 3, 5,border_mode='valid',subsample=(1, 1)))
model.add(BatchNormalization())
model.add(Activation(acti))
#model.add(Dropout(0.5))
# 7*276*32
model.add(Convolution2D(48, 3, 5,border_mode='valid',subsample=(1, 1)))
model.add(BatchNormalization())
model.add(Activation(acti))
#model.add(Dropout(0.5))
# 5*272*48
#dummy_input = np.ones((32, 13, 288, 9))
#preds = model.predict(dummy_input)
#print(preds.shape)
model.add(Deconvolution2D(32, 3, 5, output_shape=(None, 7, 276, 32),border_mode='valid'))
model.add(BatchNormalization())
model.add(Activation(acti))
#model.add(Dropout(0.5))
model.add(Deconvolution2D(24, 3, 5, output_shape=(None, 9, 280, 24),border_mode='valid'))
model.add(BatchNormalization())
model.add(Activation(acti))
#model.add(Dropout(0.5))
model.add(Deconvolution2D(16, 3, 5, output_shape=(None, 11, 284, 16),border_mode='valid'))
model.add(BatchNormalization())
model.add(Activation(acti))
#model.add(Dropout(0.5))

model.add(Deconvolution2D(1, 5, 5, output_shape=(None, 15, 288, 1),border_mode='valid'))
model.add(Activation('linear'))

ada = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mse', optimizer=ada)

In [7]:
# Structure 3
model = Sequential()
# 13*288*9
model.add(Convolution2D(16, 3, 5,input_shape=(13, 288, 9),border_mode='valid'))
model.add(BatchNormalization())
model.add(Activation('relu'))
# 11*284*16
model.add(Convolution2D(32, 3, 5,border_mode='valid'))
model.add(BatchNormalization())
model.add(Activation('relu'))
# 9*280*24
model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(60480))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Reshape((9,280,24)))
model.add(Deconvolution2D(16, 3, 5, output_shape=(None, 11, 284,16),border_mode='valid'))
model.add(BatchNormalization())
model.add(Activation('relu'))
# 11*284*16 
model.add(Deconvolution2D(1, 5, 5, output_shape=(None, 15, 288, 1),border_mode='valid'))
model.add(Activation('linear'))
ada = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mse', optimizer=ada)

In [8]:

history = model.fit(X_train, y_train, batch_size=128, nb_epoch=500, validation_split=0.1,shuffle=True)

Train on 311 samples, validate on 35 samples
Epoch 1/500
311/311 [==============================] - 13s - loss: 4767.1451 - val_loss: 4878.4194
Epoch 2/500
311/311 [==============================] - 11s - loss: 4725.9295 - val_loss: 4877.8618
Epoch 3/500
311/311 [==============================] - 11s - loss: 4721.9985 - val_loss: 4877.5308
Epoch 4/500
311/311 [==============================] - 11s - loss: 4716.2944 - val_loss: 4876.6289
Epoch 5/500
311/311 [==============================] - 11s - loss: 4707.9053 - val_loss: 4875.0444
Epoch 6/500
311/311 [==============================] - 11s - loss: 4695.9351 - val_loss: 4872.3110
Epoch 7/500
311/311 [==============================] - 11s - loss: 4679.5247 - val_loss: 4867.9453
Epoch 8/500
311/311 [==============================] - 12s - loss: 4658.4830 - val_loss: 4862.0332
Epoch 9/500
311/311 [==============================] - 12s - loss: 4632.6289 - val_loss: 4852.6001
Epoch 10/500
311/311 [==============================] - 12s - lo

In [ ]:
# check model predictions

randnumber = np.random.randint(len(data))
print(data['X'][randnumber])
sample_raw = X_train[randnumber,:,:,:]
sample = sample_raw.reshape((1,)+sample_raw.shape)

speed_prediction = model.predict(sample)
speed_prediction = speed_prediction[0,:,:,0]
speed_true = y_train[randnumber,:,:,0]

from pylab import *
from matplotlib import gridspec
cdict = {'red': ((0.0, 0.0, 0.0),
                 (0.125, 165/255, 165/255),
                 (0.25, 1.0, 1.0),
                 (0.5625, 1.0, 1.0),
                 (1.0, 0.0, 0.0)),
         'green': ((0.0, 0.0, 0.0),
                   (0.25, 0.0, 0.0),
                   (0.5625, 1.0, 1.0),
                   (1.0, 1.0, 1.0)),
         'blue': ((0.0, 0.0, 0.0),
                  (0.5, 0.0, 0.0),
                  (1.0, 0.0, 0.0))}
my_cmap = matplotlib.colors.LinearSegmentedColormap('my_colormap',cdict,256)

plt.figure(figsize=(12,4))
gs = gridspec.GridSpec(3, 3,hspace=0,wspace=0) 
for i in range(9):
    plt.subplot(gs[i])
    plt.pcolor(sample_raw[:,:,i],cmap=plt.cm.Blues)
plt.figure(figsize=(12,4))
gs = gridspec.GridSpec(1, 2, width_ratios=[5,6]) 
gs.update(left=0,right=1,hspace=0,wspace=0)
plt.subplot(gs[0])
plt.pcolor(speed_true,cmap=my_cmap)
plt.subplot(gs[1])
plt.pcolor(speed_prediction,cmap=my_cmap)
colorbar()

In [9]:
# save model

from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("structure1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("structure1.h5")

In [8]:
# load pre-trained model

json_file = open('structure1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("structure1.h5")
model.compile(loss='mse', optimizer='adam')

In [ ]:
# later...

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print "%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100)